In [8]:
from __future__ import print_function

In [9]:
import keras

In [10]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
import numpy as np

In [4]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/temp/data/", one_hot=True)
x_train = mnist.train.images[:int(55000*0.9)] - 0.5  # Returns np.array
y_train = mnist.train.labels[:int(55000*0.9)]
x_val = mnist.train.images[int(55000*0.9):]-0.5
y_val = mnist.train.labels[int(55000*0.9):]
x_test = mnist.test.images - 0.5  # Returns np.array
y_test = mnist.test.labels
# x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
# x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
# x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Extracting /temp/data/train-images-idx3-ubyte.gz
Extracting /temp/data/train-labels-idx1-ubyte.gz
Extracting /temp/data/t10k-images-idx3-ubyte.gz
Extracting /temp/data/t10k-labels-idx1-ubyte.gz
x_train shape: (49500, 784)
y_train shape: (49500, 10)
49500 train samples
10000 test samples


In [6]:
x_train[0].shape

(784,)

In [19]:
# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Reshape((28, 28, 1), input_shape=(784,)))
model.add(Conv2D(32, kernel_size=(5, 5),
                 padding='same',
                 activation='relu'))
#                  kernel_initializer=None,
#                  input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(64, kernel_size=(5, 5),
                 padding='same',
                 activation='relu'))
#                  kernel_initializer=None))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.SGD(lr=0.001),
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto') 
checkpoint = keras.callbacks.ModelCheckpoint("weights.{epoch:02d}-{val_loss:.2f}_1.hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          callbacks=[earlyStopping, checkpoint], 
          validation_data=(x_val, y_val),
          shuffle = False)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 49500 samples, validate on 5500 samples
Epoch 1/100
49500/49500 [==============================] - 130s 3ms/step - loss: 0.1893 - acc: 0.9422 - val_loss: 0.1865 - val_acc: 0.9393
Epoch 2/100
49500/49500 [==============================] - 129s 3ms/step - loss: 0.0510 - acc: 0.9845 - val_loss: 0.1019 - val_acc: 0.9687
Epoch 3/100
49500/49500 [==============================] - 130s 3ms/step - loss: 0.0346 - acc: 0.9891 - val_loss: 0.0705 - val_acc: 0.9785
Epoch 4/100
49500/49500 [==============================] - 130s 3ms/step - loss: 0.0264 - acc: 0.9916 - val_loss: 0.0994 - val_acc: 0.9684
Test loss: 0.03479329680593218
Test accuracy: 0.9888


In [15]:
model.save('model_keras_adam.h5')

In [16]:
from keras.models import load_model

In [21]:
model_new = load_model("weights.03-0.07_1.hdf5")

In [22]:
score = model_new.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.029802681393036618
Test accuracy: 0.9897


In [23]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
#         output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [24]:
frozen_graph = freeze_session(K.get_session(), output_names=[model_new.output.op.name])

INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.


In [25]:
tf.train.write_graph(frozen_graph,".", "model_keras2tf_adam.pb", as_text=False)

'.\\model_keras2tf_adam.pb'

In [14]:
[v.op.name for v in tf.global_variables()]

['conv2d_2/kernel',
 'conv2d_2/bias',
 'conv2d_3/kernel',
 'conv2d_3/bias',
 'dense_1/kernel',
 'dense_1/bias',
 'dense_2/kernel',
 'dense_2/bias',
 'Adam/iterations',
 'Adam/lr',
 'Adam/beta_1',
 'Adam/beta_2',
 'Adam/decay',
 'training/Adam/Variable',
 'training/Adam/Variable_1',
 'training/Adam/Variable_2',
 'training/Adam/Variable_3',
 'training/Adam/Variable_4',
 'training/Adam/Variable_5',
 'training/Adam/Variable_6',
 'training/Adam/Variable_7',
 'training/Adam/Variable_8',
 'training/Adam/Variable_9',
 'training/Adam/Variable_10',
 'training/Adam/Variable_11',
 'training/Adam/Variable_12',
 'training/Adam/Variable_13',
 'training/Adam/Variable_14',
 'training/Adam/Variable_15',
 'training/Adam/Variable_16',
 'training/Adam/Variable_17',
 'training/Adam/Variable_18',
 'training/Adam/Variable_19',
 'training/Adam/Variable_20',
 'training/Adam/Variable_21',
 'training/Adam/Variable_22',
 'training/Adam/Variable_23']

In [14]:
!pip install h5py